# 네이버 블로그 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
#from selenium import webdriver

In [2]:
import time
import random
import datetime
import re
#import pickle

## blog URL

In [3]:
def get_blogs(url):
    results = []
    urls = []
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    lis = soup.find_all('li', attrs={'class':'sh_blog_top', 'id':re.compile('sp_blog')})
    
    for li in lis:

        title = li.find('a', attrs={'class':re.compile(r'_sp_each_title')}).get('title').strip()
        if li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).text:
            naver_url = li.find('a', attrs={'class':re.compile(r'_sp_each_url')}).get('href').strip()
#        pub = li.find('span', attrs={'class':'_sp_each_source'}).text.strip()
#        date = li.find('span', attrs={'class':'bar'}).next_sibling.strip()
        results.append([title, naver_url])
        urls.append(naver_url)
        
    return urls

## real data URL

In [4]:
def get_realdata_url_naverblog(url):
    """
    네이버 블로그 포스트의 실제 데이터가 들어있는 URL을 반환해줌.
    보통 3가지 경우가 있는 것 같음.
    <지금까지 찾은 경우의 수>
    1. id='screenFrame'이 담긴 페이지에서 얻은 URL -> (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    2. (base_url) + (id='mainFrame'이 담긴 페이지에서 얻은 URL) -> 실제 데이터가 담긴 URL
    3. 검색해서 처음 얻은 URL이 실제 데이터가 들어가 있는 URL임
    """
    base_url = "http://blog.naver.com"
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    
    if soup.find(id='screenFrame'):
        # id = 'screenFrame'이 존재하는 경우 (id: screenFrame URL에서 -> id: mainFrame URL로 넘김)
        url = soup.find(id='screenFrame').get('src')
        r = requests.get(url)
        soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')
    if soup.find(id='mainFrame'):
        # id = 'mainFrame'이 존재하는 경우 (id: mainFrame URL에서 -> id: real data URL로 넘김)
        url = base_url + soup.find(id='mainFrame').get('src')
    return url   

## Title

In [5]:
def get_title_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 제목을 반환해줌.
    """
    try:
        #블로그 포스트 제목 추출
        title = soup.find('meta', attrs={'property':'og:title'}).get('content')
    except:
        title = 'NA'
    return title

## Datetime

In [6]:
def det_publish_datetime_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 작성일을 반환해줌.
    """
    try:
        #블로그 작성일 추출
        date_tag = soup.find('span', attrs={'class':'se_publishDate pcol2'})
        if date_tag:
            # <span> Tag의 se_publishDate pcol2 클래스 이름으로 작성일 추출이 가능한 경우
            date_str = date_tag.get_text()
        
        else:
            # <p> Tag의 date fil5 pcol2 _postAddDate 클래스 이름으로 작성일 추출이 가능한 경우
            date_tag = soup.find('p', attrs={'class' : 'date fil5 pcol2 _postAddDate'})
            if date_tag:
                date_str = date_tag.get_text()  
        try:
            #블로그 작성시간이 년, 월, 일로 되어있을 경우
            publish_datetime = datetime.datetime.strptime(date_str, '%Y. %m. %d. %H:%M')
        except:
            #블로그 작성시간이 N시간 전, 방금전 이렇게 표시될 경우
            publish_datetime = str(datetime.datetime.now()).split('.')[0] + ' ' + date_str
    except:
        publish_datetime = 'NA'
    return publish_datetime

## Text

In [7]:
def get_texts_naverblog(soup):
    """
    BeautifulSoup 객체를 받아서 블로그 포스트의 본문 내용을 반환해줌.
    """
    try:
        #블로그 본문 추출
        #보통은 <span> Tag에, id: 'SE'로 시작하는 곳에 본문이 담겨있음
        temp = soup.findAll("span", attrs= {'id': re.compile('^SE')})
        
        texts = []
        for t in temp:
            temp_text = t.get_text()
            if temp_text != '\u200b':
                texts.append(temp_text)
                    
        if not texts:
            # 블로그의 본문 텍스트가 전혀 추출되지 않았을 경우에 한번 더 시도해본다.
            # <p class="se_textarea"> 내에 존재하는 모든 본문 텍스트를 가져옴
            """
            # <p class="se_textarea"> 내의 <span> Tag에 본문이 들어가 있을 경우 
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                for span in p.findAll("span"):
                    temp_text = span.get_text()
                    if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
            """
            for p in soup.findAll("p", attrs={'class': "se_textarea"}):
                temp_text = p.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
                        
        if not texts:
            # 위의 두 가지 방법으로 본문 텍스트 추출을 시도했음에도 텍스트가 전혀 추출되지 않을 경우
            # <div> Tag 내의 id 값이 'postViewArea'이고,
            # 다시 그 태그의 내부에 존재하는 <div> Tag들 속에 존재하는 모든 본문 텍스트를 가져옴
            for div in soup.find('div', attrs={'id': 'postViewArea'}).find('div'):
                temp_text = div.get_text()
                if temp_text:
                        # temp_text에 '' 이렇게 빈 문자가 들어가있는 경우를 제외해주기 위함
                        texts.append(temp_text)
        
        texts = "\n".join(texts)
    except:
        texts = 'NA'
    return texts

##  naver URL 구성
* where = post(블로그)<br/>
* query = <br/>
* st = sim(관련도순), date(최신순) <br/>
* date_from = <br/>
* date_to = <br/>
* date_option = 0(전체), 2,3,4,5,6,7(1일,1주,1개월,6개월,1년), 8(직접입력) <br/>
* srchby = all(전체), title(제목)<br/>

## selenium으로 blog url 불러오기

In [8]:
from tqdm import tqdm_notebook
import pandas as pd

In [9]:
date_pd = pd.date_range('2016-08-01', '2019-04-30').strftime("%Y%m%d") # skt 누구 - 2016.09
date_pd

Index(['20160801', '20160802', '20160803', '20160804', '20160805', '20160806',
       '20160807', '20160808', '20160809', '20160810',
       ...
       '20190421', '20190422', '20190423', '20190424', '20190425', '20190426',
       '20190427', '20190428', '20190429', '20190430'],
      dtype='object', length=1003)

In [10]:
query = '"nugu"%20스피커'

#pickle_name = q+'_'+startDate+'_'+endDate+'.p'
total_url = []

for date in tqdm_notebook(date_pd):
    for page in range(1,100):
        try:
            time.sleep(0.01)

            url = 'https://search.naver.com/search.naver?date_from='+str(date)+'&date_option=8&date_to='+str(date)+'&dup_remove=1&nso=p%3Afrom'+str(date)+'to'+str(date)+'&query='+query+'&where=post&start='+str(page)

            results = get_blogs(url)

            if results[0] in total_url:
                break

            total_url = total_url + results
        except:
            break     

#pickle.dump(total_results.open(pickle_name,'wb'))

## Get real_url, title, datetime, text

In [20]:
with open('nugu_speaker.txt', 'w', encoding = 'utf8') as f:
    title_list = []
    publish_datetime_list = []
    text_list = []
    url_list = []
    
    for full_url in tqdm_notebook(total_url):
        
        try:       
            # 실제 데이터가 들어있는 url 추출
            real_url = get_realdata_url_naverblog(full_url)
            r = requests.get(real_url)
            soup = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

            # 블로그 포스트 제목 추출
            title = get_title_naverblog(soup)
            if title in title_list:
                continue
            title_list.append(title)

            # 블로그 작성일 추출
            publish_datetime = det_publish_datetime_naverblog(soup)
            publish_datetime_list.append(publish_datetime)

            # 블로그 텍스트 추출
            texts = get_texts_naverblog(soup)
            text_list.append(texts)

            # 중복 제거된 url
            url_list.append(real_url)

            #content = ' '.join(texts)
            f.write(texts+'\n') #결과를 text 파일에 저장합니다.
            
        except:
            continue

## dataframe.to_csv

In [23]:
import pandas as pd
df = pd.DataFrame(data={'title': title_list, 
                        'publish_datetime': publish_datetime_list,
                        'text': text_list, 
                        'url': url_list})

In [24]:
#df = df[['title', 'publish_datetime', 'text', 'url']]
print(df.shape)
df.head()

(3087, 4)


,title,publish_datetime,text,url
0,"SKT 음성인식 스피커 ""NUGU (누구)"" 출시 (like 아마존에코, Amazo...",2016-08-31 11:02:00,NA,http://blog.naver.com/PostView.nhn?blogId=zoop...
1,"로봇 비서?…SKT, 음성인식AI ‘누구’ 공개",2016-08-31 16:09:00,1일부터 서비스…인공지능 대중화 시대 선언\n“비 오는 초가을 날씨에 어울리는 노래...,http://blog.naver.com/PostView.nhn?blogId=duop...
2,[2016.8.31 뉴스클리핑] ▲'핀테크 고속도로' 개통…금융권 공동 오픈플랫폼 ...,2016-08-31 12:26:00,세계 첫 사례…핀테크 기업들 빠른 서비스 개발 가능 핀테크 기업이 새로운 서비스를 ...,http://blog.naver.com/PostView.nhn?blogId=kiss...
3,NA,NA,NA,http://blog.daum.net/dandakhan/16619935
4,"“ ‘누구’ 신나는 음악좀 틀어줘” SKT, 생활속 AI 로봇 서비스....",2016-08-31 21:19:00,NA,http://blog.naver.com/PostView.nhn?blogId=chas...


In [25]:
df.to_csv('./nugu_speaker_blog.csv',sep=',',encoding='UTF-16')

In [26]:
pd.read_csv('./nugu_speaker_blog.csv', encoding='UTF-16',index_col=0)

,title,publish_datetime,text,url
0,"SKT 음성인식 스피커 ""NUGU (누구)"" 출시 (like 아마존에코, Amazo...",2016-08-31 11:02:00,NaN,http://blog.naver.com/PostView.nhn?blogId=zoop...
1,"로봇 비서?…SKT, 음성인식AI ‘누구’ 공개",2016-08-31 16:09:00,1일부터 서비스…인공지능 대중화 시대 선언\n“비 오는 초가을 날씨에 어울리는 노래...,http://blog.naver.com/PostView.nhn?blogId=duop...
2,[2016.8.31 뉴스클리핑] ▲'핀테크 고속도로' 개통…금융권 공동 오픈플랫폼 ...,2016-08-31 12:26:00,세계 첫 사례…핀테크 기업들 빠른 서비스 개발 가능 핀테크 기업이 새로운 서비스를 ...,http://blog.naver.com/PostView.nhn?blogId=kiss...
3,NaN,NaN,NaN,http://blog.daum.net/dandakhan/16619935
4,"“ ‘누구’ 신나는 음악좀 틀어줘” SKT, 생활속 AI 로봇 서비스....",2016-08-31 21:19:00,NaN,http://blog.naver.com/PostView.nhn?blogId=chas...
5,"SK텔레콤 '누구(NUGU)', AI 대중화 시대 선언",2016-08-31 11:01:00,NaN,http://blog.naver.com/PostView.nhn?blogId=jinm...
6,"SK텔레콤, 인공지능 서비스 ‘누구(NUGU)’ 출시",2016-08-31 10:32:00,NaN,http://blog.naver.com/PostView.nhn?blogId=news...
7,NUGU 너 누구니? 인공지능 음성인식 디바이스,2016-09-01 16:12:00,NaN,http://blog.naver.com/PostView.nhn?blogId=movi...
8,SKT 인공지능(?) 음성인식디바이스 NUGU 누구,2016-09-01 17:25:00,NaN,http://blog.naver.com/PostView.nhn?blogId=akoh...
9,누구와도 말 통하는 SK표 인공지능 비서,2016-09-01 07:50:00,기사입력 2016.09.01 오전 12:04최종수정 2016.09.01 오전 6:2...,http://blog.naver.com/PostView.nhn?blogId=jsym...
